# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Transfer-Learning" data-toc-modified-id="Transfer-Learning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Transfer Learning</a></div><div class="lev1 toc-item"><a href="#save_bottlebeck_features()" data-toc-modified-id="save_bottlebeck_features()-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>save_bottlebeck_features()</a></div><div class="lev1 toc-item"><a href="#또는-직접-불러-오기" data-toc-modified-id="또는-직접-불러-오기-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>또는 직접 불러 오기</a></div><div class="lev1 toc-item"><a href="#train_top_model()" data-toc-modified-id="train_top_model()-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>train_top_model()</a></div><div class="lev1 toc-item"><a href="#Transfer-Learning-&amp;-fine-tuning" data-toc-modified-id="Transfer-Learning-&amp;-fine-tuning-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Transfer Learning &amp; fine tuning</a></div><div class="lev2 toc-item"><a href="#방법-1-:-Bottleneck-Feature-사용하지-않기" data-toc-modified-id="방법-1-:-Bottleneck-Feature-사용하지-않기-51"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>방법 1 : Bottleneck Feature 사용하지 않기</a></div><div class="lev2 toc-item"><a href="#방법-2:-Bottle-Feature-사용" data-toc-modified-id="방법-2:-Bottle-Feature-사용-52"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>방법 2: Bottle Feature 사용</a></div><div class="lev1 toc-item"><a href="#학습-수행" data-toc-modified-id="학습-수행-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>학습 수행</a></div>

# Transfer Learning

![](http://i.imgur.com/1RvG7sN.jpg)

참고 자료 
- [Building powerful image classification models using very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
- [classifier_from_little_data_script_1.py](https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d)
- [classifier_from_little_data_script_2.py](https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069)
- [classifier_from_little_data_script_3.py](https://gist.github.com/fchollet/7eb39b44eb9e16e59632d25fb3119975)

[neocortex](https://github.com/neocortex/keras-transfer-learning)의 구현
- 모델 직접 정의 하여 학습 
- [03_using_bottleneck_features.ipynb](https://github.com/neocortex/keras-transfer-learning/blob/master/03_using_bottleneck_features.ipynb)
- [04_fine_tuning.ipynb](https://github.com/neocortex/keras-transfer-learning/blob/master/04_fine_tuning.ipynb)
- [05_classify_inria.ipynb](https://github.com/neocortex/keras-transfer-learning/blob/master/05_classify_inria.ipynb)

Prakash의 구현
- [Post](https://medium.com/towards-data-science/transfer-learning-using-keras-d804b2e04ef8)
- [GitHUb](https://github.com/Prakashvanapalli)

In [13]:
from __future__ import print_function
from __future__ import absolute_import

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D

from keras import applications

In [2]:
import tensorflow as tf; print("TensorFLow Version:"+str(tf.__version__))
import keras; print("Keras Version:"+str(keras.__version__))

TensorFLow Version:1.1.0
Keras Version:2.0.3


In [17]:
# dimensions of our images.

top_model_weights_path = 'fc_model.h5'
train_data_dir = 'dog-cat/train'
validation_data_dir = 'dog-cat/val'
nb_train_samples = 2222 #2000 
nb_validation_samples = 222 #800
epochs = 50
batch_size = 16

img_width, img_height = 150, 150
from keras import backend as K
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
    print("Theano : Channel First"+str(input_shape))
else:
    input_shape = (img_width, img_height, 3)
    print("TensorFlow : Channel Last"+str(input_shape))

TensorFlow : Channel Last(150, 150, 3)


# save_bottlebeck_features()
- bottleneck_features는 FC가 없는 VGG16 모델을 가지고 predict()한 값 이다.

In [27]:
# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet', input_shape=input_shape) ## TF / TH구분으로 inputshape는 지정 필요
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

# 또는 직접 불러 오기 

``` python
weights_path = 'vgg16_weights.h5'

# build the VGG16 network
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=input_shape #input_shape=(3, img_width, img_height)))

model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

```

In [22]:
datagen = ImageDataGenerator(rescale=1. / 255)


#train_data
generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

#validation_data
generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples // batch_size)
#4분 cpu12-mem20-cost0344

Found 2222 images belonging to 2 classes.
Found 222 images belonging to 2 classes.


?? bottleneck_features를 생성(predict)할때 FC가 없는 불완전한 모델(include_top=False)로 하는것이 맞는가?

In [31]:
np.save(open('bottleneck_features_train.npy', 'wb'),
    bottleneck_features_train)
np.save(open('bottleneck_features_validation.npy', 'wb'),
    bottleneck_features_validation)

#write() argument must be str, not bytes 에러시   'w' ->'wb'

# train_top_model()
- top_model_weights는 `FC레이어`만있는 모델을 학습한 결과의 weight
- 학습시 사용하는 데이터는 bottleneck_features이다. 
    - bottleneck_features는 FC가 없는 VGG16 모델을 가지고 `predict()`한 값 이다. 
    - bottleneck을 사용하는 이유는 FC만 학습 한다고 해도 모델이 크면 시간이 많이 걸리므로 학습(fit)이 아닌 빠른 예측(predict) 값을 사용

In [61]:
import numpy as np
train_data = np.load(open('bottleneck_features_train.npy', 'rb')) #'rb'추가
train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

# can't multiply sequence by non-int of type 'float' : 해결책  / -> //


In [62]:
#강제 데이터 - 라벨 수 맞춤
train_labels = train_labels[0:2190]
validation_labels = validation_labels[0:206]

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [64]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [65]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
_________________________________________________________________


In [66]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy', metrics=['accuracy'])

In [68]:
model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))

Train on 2190 samples, validate on 206 samples
Epoch 1/50
2190/2190 [==============================] - 1s - loss: 0.6972 - acc: 0.4886 - val_loss: 0.6910 - val_acc: 0.5485
Epoch 2/50
2190/2190 [==============================] - 1s - loss: 0.6934 - acc: 0.4858 - val_loss: 0.6931 - val_acc: 0.5388
Epoch 3/50
2190/2190 [==============================] - 1s - loss: 0.6932 - acc: 0.5009 - val_loss: 0.6928 - val_acc: 0.5388
Epoch 4/50
2190/2190 [==============================] - 1s - loss: 0.6931 - acc: 0.5073 - val_loss: 0.6927 - val_acc: 0.5388
Epoch 5/50
2190/2190 [==============================] - 1s - loss: 0.6931 - acc: 0.5073 - val_loss: 0.6925 - val_acc: 0.5388
Epoch 6/50
2190/2190 [==============================] - 1s - loss: 0.6931 - acc: 0.5073 - val_loss: 0.6924 - val_acc: 0.5388
Epoch 7/50
2190/2190 [==============================] - 1s - loss: 0.6931 - acc: 0.5073 - val_loss: 0.6923 - val_acc: 0.5388
Epoch 8/50
2190/2190 [==============================] - 1s - loss: 0.6931 - ac

In [70]:
model.save_weights(top_model_weights_path)

# Transfer Learning & fine tuning

## 방법 1 : Bottleneck Feature 사용하지 않기
- top_model_weights를 사용하지 않고 

In [20]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

In [21]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

In [22]:
model = Model(base_model.input, predictions)

In [23]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
#for layer in model.layers[:5]:
#    layer.trainable = False

In [24]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

> 학습 수행 부분으로 이동

## 방법 2: Bottle Feature 사용 

In [14]:
model = applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

In [15]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [18]:
top_model.load_weights(top_model_weights_path)

In [19]:
model.add(top_model)

AttributeError: 'Model' object has no attribute 'add'

---

# 학습 수행

In [25]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 2222 images belonging to 2 classes.
Found 222 images belonging to 2 classes.


In [ ]:
# Save the model according to the conditions  
# from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [26]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples
    callbacks = [checkpoint, early]
    )

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., epochs=50, validation_steps=222, steps_per_epoch=138)`
  import sys


Epoch 1/50


ValueError: Error when checking model target: expected dense_6 to have shape (None, 200) but got array with shape (16, 1)